In [ ]:
import pickle
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import pickle
from gensim.models import Word2Vec, FastText  # For Word2Vec model
import random
import numpy as np
import utils
# ! pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device is', device)


NUM_CLASSES=23
BATCH_SIZE=256
EPOCHS=10
HIDDEN_SIZE = 768
VECTOR_SIZE = 768
TRAINING_SIZE =500000
NUM_LAYERS = 1
DROP_OUT=0

In [ ]:
dev_data=utils.read_data("../data/fixed_PIZZA_dev.json")
dev_corpus, dev_top = utils.get_dev_dataset(dev_data)
ner_dev_labels,is_dev_labels,dev_as_tokenized_string=utils.label_complete_dev(dev_corpus, dev_top)
print('Parse Dev Data Done')

In [47]:
class LargeWordRNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim,hidden_size, num_classes):
        super(LargeWordRNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_size, batch_first=True, bidirectional=True, num_layers=NUM_LAYERS, dropout=DROP_OUT)
        self.fc = nn.Linear(hidden_size * 2, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        lstm_out, _ = self.rnn(x)
        out = self.fc(lstm_out)
        return out

pickle_file_path1 = "../models/is_model.pk1"
with open(pickle_file_path1, "rb") as file1:
    is_model = pickle.load(file1)


pickle_file_path2 = "../models/word_to_index.pk1"
with open(pickle_file_path2, "rb") as file2:
    word_to_index = pickle.load(file2)

In [48]:
class TestLargeDataset(Dataset):
    def __init__(self, data):
        self.data = data 
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]

def test_collate_fn(batch):
    sequences = batch
    embeddings=[]
    for seq in sequences:
        x=[]
        for token in seq:
            if token not in word_to_index:
                x.append(word_to_index['i'])
            else:
                x.append(word_to_index[token])
        embeddings.append(x)
    sequences=embeddings
    sequences = [torch.tensor(seq) for seq in sequences]
    padded_sequences = pad_sequence(sequences, batch_first=True)
    return padded_sequences
dev_dataset = TestLargeDataset(dev_as_tokenized_string)
dataloader = DataLoader(dev_dataset, batch_size=1, collate_fn=test_collate_fn, shuffle=False, num_workers=0)

In [ ]:
is_model_output=[]

is_model.eval()
with torch.no_grad():
    for padded_sequences in dataloader:
        labels = []
        padded_sequences=padded_sequences.to(device)
        outputs = is_model(padded_sequences)
        intent_to_num = {"I_PIZZAORDER": 0, "I_DRINKORDER": 1, "I_COMPLEX_TOPPING": 2, "B_PIZZAORDER": 3, "B_DRINKORDER": 4, "B_COMPLEX_TOPPING": 5, "NONE": 6}
        for i, out in enumerate(outputs[0]):
            num = torch.argmax(out).int().item()
            labels.append(num)
        is_model_output.append(labels)

confusion_matrix, accuracy,exact_accuracy=utils.calc_accuracy(dev_corpus, is_model_output, is_dev_labels, 7)
print("word accuracy:",accuracy*100,"EM",exact_accuracy*100)
is_model_output = utils.intent_post_processing(dev_corpus,is_model_output)
confusion_matrix, accuracy,exact_accuracy=utils.calc_accuracy(dev_corpus, is_model_output, is_dev_labels, 7)
print("word accuracy:",accuracy*100,"EM",exact_accuracy*100)